## 1. Data Load

In [1]:
import pandas as pd
import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split
import numpy as np
from configs import DEFINES

PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

MARKER = [PAD, STD, END, UNK]

"""
load_data(): 데이터를 읽고 트레이닝 셋과 테스트 셋으로 분리
train_q: 트레이닝용 질문 데이터
train_a: 트레이닝용 대답 데이터
test_q: 테스트용 질문 데이터
test_a: 테스트용 대답 데이터
output: train_q, train_a, test_q, test_a
"""

def load_data():

    data_df = pd.read_csv(DEFINES.data_path, header=0)
    question, answer = list(data_df['Q']), list(data_df['A'])
    train_q, test_q, train_a, test_a = train_test_split(question, answer, test_size=0.33,
                                                                        random_state=42)
    return train_q, train_a, test_q, test_a


In [2]:
data_out_path = os.path.join(os.getcwd(), './')
os.makedirs(data_out_path, exist_ok=True)

train_q, train_a, test_q, test_a = load_data()

## 2. Tokenizing

In [3]:
"""
prepro_noise_canceling(data): 
텍스트 데이터에 정규화를 사용하여 ([~.,!?\"':;)(]) 제거
output: ([~.,!?\"':;)(]) 제거된 텍스트 데이터
"""

def prepro_noise_canceling(data):

    CHANGE_FILTER = re.compile("([~.,!?\"':;)(])")
    #  리스트를 생성합니다.
    result_data = list()

    for seq in data:
        
        seq = re.sub(CHANGE_FILTER, "", seq)
        result_data.append(seq)

    return result_data


"""
tokenizing_data(data): 텍스트 데이터들 토크나이징
input:
data: 텍스트 데이터 행렬

데이터 노이즈 처리 함수 처리 ->
띄어쓰기 단위로 나누기 ->
띄어진 단어들 벡터 형성
output: [token1, token2, ...]
"""

def tokenizing_data(data):
    
    doc = []
    data = prepro_noise_canceling(data)
    for sentence in data:
        words = []
        for word in sentence.split(): 
            words.append(word)
        doc.append(words)
    return doc

from konlpy.tag import Okt


In [4]:
token_train_q = tokenizing_data(train_q)
token_train_a = tokenizing_data(train_a)
token_test_q = tokenizing_data(test_q)
token_test_a = tokenizing_data(test_a)

## 3. Vocabulary Dictionary

In [5]:
"""
load_voc(): 단어 사전 vocabularyData.voc를 생성하고 단어와 인덱스 관계를 출력

data_path에서 csv 파일 데이터 읽어오기 ->
읽어온 데이터 노이즈 캔슬링 ->
데이터 토큰화 ->
모든 토큰을 set을 통하여 중복없는 토큰 list 생성 ->
패딩 데이터 추가 ->
vocabularyData.voc 저장 ->
make_voc를 사용하여 각 토큰에 해당되는 인덱스와 인덱스에 해당되는 토큰 데이터 생성

output: {token1:index1, token2:index2,...}, {index1:token1, index2:token2,...}, length of voc
"""

def load_voc(token_train_q, token_train_a):

    voc_list = []
    if (not (os.path.exists(DEFINES.vocabulary_path))):

        words = []

        words.extend(token_train_q)
        words.extend(token_train_a)

        s = set()
        for word in words:
            for token in word:
                s.add(token)
        
        words = list(s)
       
        words[:0] = MARKER
        
        with open(DEFINES.vocabulary_path, 'w', encoding='utf-8') as voc_file:
            for word in words:
                voc_file.write(word + '\n')

    with open(DEFINES.vocabulary_path, 'r', encoding='utf-8') as voc_file:
        for line in voc_file:
            voc_list.append(line.strip())

    char2idx, idx2char = make_voc(voc_list)

    return char2idx, idx2char, len(char2idx)

"""
make_voc(voc_list): 사전 리스트를 받아 인덱스와 토큰의 dictionary를 생성

output: {token1:index1, token2:index2,...}, {index1:token1, index2:token2,...}
"""

def make_voc(voc_list):

    char2idx = {char: idx for idx, char in enumerate(voc_list)}
    idx2char = {idx: char for idx, char in enumerate(voc_list)}
    return char2idx, idx2char

In [6]:
char2idx, idx2char, vocabulary_length = load_voc(token_train_q, token_train_a)

## 4. Input for Encoder 

In [7]:

"""
enc_processing(tokens, dictionary): 인코더용 입력값 생성 함수
텍스트 데이터 -> 인덱스 벡터화 및 길이
input:
tokens: 텍스트 문장들 데이터
dictionary: 값이 인덱스인 단어 사전

tokens 값 노이즈 캔슬링 ->
문장 단위로 나누기 ->
문장을 토큰 단위로 나누기 ->
dictionary를 활용하여 토큰 인덱스화 ->
dictionary 없는 토큰의 경우 unk값으로 대체 ->
기준 문장 길이 보다 크게 된다면 뒤의 토큰 자르기 ->
기준 문장 길이에 맞게 남은 공간에 padding ->
문장 인덱스와 문장 길이 계산
output: 넘파이 문장 인덱스 벡터, 문장 길이 
"""

def enc_input(tokens, dictionary):
    
    seq_input_index = []
    
    seq_len = []

    for seq in tokens:
        
        seq_index = []

        for token in seq:

            if dictionary.get(token) is not None:
                seq_index.extend([dictionary[token]])

            else:
                seq_index.extend([dictionary[UNK]])
        if len(seq_index) > DEFINES.max_sequence_length:
            seq_index = seq_index[:DEFINES.max_sequence_length]

        seq_len.append(len(seq_index))

        seq_index += (DEFINES.max_sequence_length - len(seq_index)) * [dictionary[PAD]]

        seq_input_index.append(seq_index)

    return np.asarray(seq_input_index)


In [8]:
train_input_enc = enc_input(token_train_q, char2idx)

## 5. Input for Decoder

In [15]:
"""
dec_input(tokens, dictionary): 디코더용 입력값 생성 함수
텍스트 데이터 -> 인덱스 벡터화 및 길이
input:
tokens: 텍스트 문장들 데이터
dictionary: 값이 인덱스인 단어 사전

tokens 값 노이즈 캔슬링 ->
문장 단위로 나누기 ->
문장을 토큰 단위로 나누기 ->
dictionary를 활용하여 토큰 인덱스화 ->
인덱스 앞에 STD 추가 ->
기준 문장 길이 보다 크게 된다면 뒤의 토큰 자르기 ->
기준 문장 길이에 맞게 남은 공간에 padding ->
문장 인덱스와 문장 길이 계산
output: 넘파이 문장 인덱스 벡터, 문장 길이 
"""


def dec_input(tokens, dictionary):

    seq_input_index = []

    seq_len = []

    for seq in tokens:

        seq_index = []

        for token in seq:
            if dictionary.get(token) is not None:
                seq_index.extend([dictionary[token]])
            else:
                seq_index.extend([dictionary[UNK]])
        seq_index = [dictionary[STD]] + seq_index
        
        if len(seq_index) > DEFINES.max_sequence_length:
            seq_index = seq_index[:DEFINES.max_sequence_length]
        seq_len.append(len(seq_index))
        seq_index += (DEFINES.max_sequence_length - len(seq_index)) * [dictionary[PAD]]
        seq_input_index.append(seq_index)

    return np.asarray(seq_input_index)

"""
dec_target_processing(value, dictionary): 디코더용 입력값 생성 함수
텍스트 데이터 -> 인덱스 벡터화 및 길이
input:
value: 텍스트 문장들 데이터
dictionary: 값이 인덱스인 단어 사전

value 값 노이즈 캔슬링 ->
문장 단위로 나누기 ->
문장을 토큰 단위로 나누기 ->
dictionary를 활용하여 토큰 인덱스화 ->
기준 문장 길이 보다 크게 된다면 뒤의 토큰 자르기 ->
인덱스 뒤에 END 추가 ->
기준 문장 길이에 맞게 남은 공간에 padding ->
문장 인덱스와 문장 길이 계산
output: 넘파이 문장 인덱스 벡터, 문장 길이 
"""

def dec_target(tokens, dictionary):
    
    seq_target_index = []
    
    for seq in tokens:
        
        seq_index = []
        
        for token in seq:
           
            if dictionary.get(token) is not None:
                seq_index.extend([dictionary[token]])
            
            else:
                seq_index.extend([dictionary[UNK]])
        
        if len(seq_index) >= DEFINES.max_sequence_length:
            seq_index = seq_index[:DEFINES.max_sequence_length - 1] + [dictionary[END]]
        else:
            seq_index += [dictionary[END]]
        
        seq_index += (DEFINES.max_sequence_length - len(seq_index)) * [dictionary[PAD]]
        
        seq_target_index.append(seq_index)
    
    return np.asarray(seq_target_index)


In [17]:
train_input_dec = dec_input(token_train_a, char2idx)
train_target_dec = dec_target(token_train_a, char2idx)
test_input_enc = enc_input(token_test_q, char2idx)
test_input_dec = dec_input(token_test_a, char2idx)
test_target_dec = dec_target(token_test_a, char2idx)

## 6. Model Setting

In [18]:
check_point_path = os.path.join(os.getcwd(), DEFINES.check_point_path)
os.makedirs(check_point_path, exist_ok=True)

In [19]:
import model as ml

# 에스티메이터 구성
classifier = tf.estimator.Estimator(
        model_fn=ml.model, # 모델 등록한다.
        model_dir=DEFINES.check_point_path, 
        params={
            'hidden_size': DEFINES.hidden_size, 
            'layer_size': DEFINES.layer_size, 
            'learning_rate': DEFINES.learning_rate, 
            'vocabulary_length': vocabulary_length, 
            'embedding_size': DEFINES.embedding_size, 
        })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/check_point', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F0D8C8B588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
# input과 output dictionary를 만드는 함수
def in_out_dict(input, output, target):
    features = {"input": input, "output": output}
    return features, target

# 학습용 배치 데이터를 만드는 함수
def train_input_fn(train_input_enc, train_input_dec, train_target_dec, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((train_input_enc, train_input_dec, train_target_dec))
    dataset = dataset.shuffle(buffer_size=len(train_input_enc))
    assert batch_size is not None, "train batchSize must not be None"
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(in_out_dict)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()


# 평가용 배치 데이터를 만드는 함수
def test_input_fn(test_input_enc, test_input_dec, test_target_dec, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((test_input_enc, test_input_dec, test_target_dec))
    dataset = dataset.shuffle(buffer_size=len(test_input_enc))
    assert batch_size is not None, "eval batchSize must not be None"
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(in_out_dict)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

## 7. Training

In [ ]:
classifier.train(input_fn=lambda: train_input_fn(
        train_input_enc, train_input_dec, train_target_dec, DEFINES.batch_size), steps=DEFINES.train_steps)

## 8. Accuracy

In [21]:
test_result = classifier.evaluate(input_fn=lambda: test_input_fn(
        test_input_enc, test_input_dec, test_target_dec, DEFINES.batch_size))

print('\nEVAL set accuracy: {accuracy:0.3f}\n'.format(**test_result))

Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation 

## 9. Prediction

In [22]:
predic_input_enc = enc_input(["가끔 궁금해"], char2idx)
predic_input_dec = dec_input([""], char2idx)
predic_target_dec = dec_target([""], char2idx)

In [23]:
"""
pred_next_string(value, dictionary): 예측용 단어 인덱스를 문장으로 변환하는 함수
input:
value: 텍스트 문장들 데이터
dictionary: 값이 인덱스인 단어 사전

단어 사전을 이용하여 각 인덱스의 해당되는 단어로 문장 형성->
패딩 값은 스페이스 처리 ->
END가 들어가면 출력완료 값 = True

output: 변환된 문장, 출력완료 boolean값
"""

def pred_next_string(tokens, dictionary):

    sentence_string = []

    for token in tokens:
        sentence_string = [dictionary[index] for index in token['indexs']]
    print(sentence_string)
    answer = ""

    for word in sentence_string:
        if word not in PAD and word not in END:
            answer += word
            answer += " "

    print(answer)
    return answer

In [24]:
predictions = classifier.predict(
        input_fn=lambda: test_input_fn(predic_input_enc, predic_input_dec, predic_target_dec, DEFINES.batch_size))

answer = pred_next_string(predictions, idx2char)

print("answer: ", answer)
    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/check_point\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
['0칼로리', '병원', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
0칼로리 병원 
answer:  0칼로리 병원 


## 10. Evaluation

In [18]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge import Rouge

# bleu score 계산 함수
def bleu_compute(ground_truth_utter, answer_sample):
    ground_truth_utter_list = ground_truth_utter.split()
    answer_sample_list = answer_sample.split()
    return sentence_bleu([ground_truth_utter_list], answer_sample_list, smoothing_function=SmoothingFunction().method7,
                         weights=[1./3, 1./3, 1./3])

# rouge score 계산 함수
def rouge_compute(ground_truth_utter, answer_sample):
    rouge = Rouge()
    scores = rouge.get_scores(ground_truth_utter, answer_sample)
    return np.array([scores[0]["rouge-l"]["p"], scores[0]["rouge-l"]["r"], scores[0]["rouge-l"]["f"]])

In [21]:
print("Bleu score: ", bleu_compute("그 사람도 그럴 거예요", answer))
print("Rouge score: ", rouge_compute("그 사람도 그럴 거예요", answer))

Bleu score:  1.013978363419326
Rouge score:  [1. 1. 1.]


In [30]:
train_q[10:20]

['너무 힘든데',
 '툭하면 헤어지자라는 말을 하는 사람',
 '헤어지신 남자분들이나 여자분들',
 '단둘이는 처음 만나는 날ㅋㅋ',
 '쌈은 뭐야',
 '그림 잘 그리고 싶다',
 '무슨마음일까',
 '신기해',
 '많이 힘드네',
 '머리 용량 초과']

In [31]:
train_a[10:20]

['조금만 더 버텨보세요.',
 '서운한 마음을 충분히 전하는게 좋겠어요.',
 '누구든 힘들어할 거예요.',
 '서로를 아는 좋은 기회가 되겠네요.',
 '썸 타다가 발전되지 못한 관계죠.',
 '학원을 다니거나 연습하면 잘할 수 있을 거예요.',
 '사람 마을은 알기 힘들어요.',
 '그게 인생이죠.',
 '그 과정을 통해 성장할 수 있을 거예요.',
 '눈을 감고 명상을 해보세요.']